



https://shiring.github.io/machine_learning/2017/04/23/lime

https://shiring.github.io/machine_learning/2017/04/23/one_r

In [1]:
#install.packages("tidyquant")

NameError: name 'install' is not defined

In [3]:
#install.packages("h2o")

Installing package into ‘/home/jon/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
also installing the dependency ‘RCurl’



Installing package into ‘/home/jon/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
Warning message:
“package ‘matick’ is not available (for R version 3.4.4)”

In [7]:
#install.packages("lime")
# from Terminal:
# sudo apt-get install libmagick++-dev

# I then installed LIME (from CRAN) with no problems.
# Yay! LIME finally installed OK :-)

Installing package into ‘/home/jon/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
also installing the dependency ‘magick’



In [1]:
# Load the following packages
library(tidyquant)  # Loads tidyverse and several other pkgs 
library(readxl)     # Super simple excel reader
library(h2o)        # Professional grade ML pkg
library(lime)       # Explain complex black-box ML models

Loading required package: lubridate

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: PerformanceAnalytics
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Loading required package: quantmod
Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.
Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.5
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date

In [2]:
# Read excel data
hr_data_raw <- read_excel(path = "data/WA_Fn-UseC_-HR-Employee-Attrition.xlsx")

In [3]:
hr_data_raw[1:10,]

Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,⋯,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,⋯,1,80,0,8,0,1,6,4,0,5
49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,⋯,4,80,1,10,3,3,10,7,1,7
37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,⋯,2,80,0,7,3,3,0,0,0,0
33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,⋯,3,80,0,8,3,3,8,7,3,0
27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,⋯,4,80,1,6,3,3,2,2,2,2
32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,⋯,3,80,0,8,2,2,7,7,3,6
59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,⋯,1,80,3,12,3,2,1,0,0,0
30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,⋯,2,80,1,1,2,3,1,0,0,0
38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,⋯,2,80,0,10,2,3,9,7,1,8
36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,⋯,2,80,2,17,3,2,7,7,7,7


In [4]:
# View first 10 rows
# hr_data_raw[1:10,] %>%
#     knitr::kable("html",caption = "First 10 rows")%>%as.character()%>%display_html()

In [5]:
hr_data <- hr_data_raw %>%
    mutate_if(is.character, as.factor) %>%
    select(Attrition, everything())

In [6]:
glimpse(hr_data)

Observations: 1,470
Variables: 35
$ Attrition                <fct> Yes, No, Yes, No, No, No, No, No, No, No, ...
$ Age                      <dbl> 41, 49, 37, 33, 27, 32, 59, 30, 38, 36, 35...
$ BusinessTravel           <fct> Travel_Rarely, Travel_Frequently, Travel_R...
$ DailyRate                <dbl> 1102, 279, 1373, 1392, 591, 1005, 1324, 13...
$ Department               <fct> Sales, Research & Development, Research & ...
$ DistanceFromHome         <dbl> 1, 8, 2, 3, 2, 2, 3, 24, 23, 27, 16, 15, 2...
$ Education                <dbl> 2, 1, 2, 4, 1, 2, 3, 1, 3, 3, 3, 2, 1, 2, ...
$ EducationField           <fct> Life Sciences, Life Sciences, Other, Life ...
$ EmployeeCount            <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
$ EmployeeNumber           <dbl> 1, 2, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, ...
$ EnvironmentSatisfaction  <dbl> 2, 3, 4, 4, 1, 4, 3, 4, 4, 3, 1, 4, 1, 2, ...
$ Gender                   <fct> Female, Male, Male, Female, Male, Male, Fe...
$ HourlyRate      

Modeling Employee Attrition

We are going to use the h2o.automl() function from the H2O platform to model employee attrition.
H2O

First, we need to initialize the Java Virtual Machine (JVM) that H2O uses locally.

In [14]:
# Initialize H2O JVM
h2o.init() 



H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/Rtmpv2FLys/h2o_jon_started_from_r.out
    /tmp/Rtmpv2FLys/h2o_jon_started_from_r.err


Starting H2O JVM and connecting: . Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         3 seconds 794 milliseconds 
    H2O cluster timezone:       America/Vancouver 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.20.0.2 
    H2O cluster version age:    2 months and 26 days  
    H2O cluster name:           H2O_started_from_R_jon_yyc570 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   2.60 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  8 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, C

In [15]:
h2o.no_progress() # Turn off output of progress bars

In [16]:
# Split data into Train/Validation/Test Sets
hr_data_h2o <- as.h2o(hr_data)

split_h2o <- h2o.splitFrame(hr_data_h2o, c(0.7, 0.15), seed = 1234 )

train_h2o <- h2o.assign(split_h2o[[1]], "train" ) # 70%
valid_h2o <- h2o.assign(split_h2o[[2]], "valid" ) # 15%
test_h2o  <- h2o.assign(split_h2o[[3]], "test" )  # 15%

In [17]:
# Set names for h2o
y <- "Attrition"
x <- setdiff(names(train_h2o), y)

Now the fun begins. We run the h2o.automl() setting the arguments it needs to run models against. For more information, see the h2o.automl documentation.

    x = x: The names of our feature columns.
    y = y: The name of our target column.
    training_frame = train_h2o: Our training set consisting of 70% of the data.
    leaderboard_frame = valid_h2o: Our validation set consisting of 15% of the data. H2O uses this to ensure the model does not overfit the data.
    max_runtime_secs = 30: We supply this to speed up H2O’s modeling. The algorithm has a large number of complex models so we want to keep things moving at the expense of some accuracy.


In [18]:
# Run the automated machine learning 
automl_models_h2o <- h2o.automl(
    x = x, 
    y = y,
    training_frame    = train_h2o,
    leaderboard_frame = valid_h2o,
    max_runtime_secs  = 30
    )

In [ ]:
# Extract leader model
automl_leader <- automl_models_h2o@leader

In [ ]:
# Predict on hold-out set, test_h2o
pred_h2o <- h2o.predict(object = automl_leader, newdata = test_h2o)

https://www.business-science.io/business/2017/09/18/hr_employee_attrition.html

In [ ]:
# Prep for performance assessment
test_performance <- test_h2o %>%
    tibble::as_tibble() %>%
    select(Attrition) %>%
    add_column(pred = as.vector(pred_h2o$predict)) %>%
    mutate_if(is.character, as.factor)
test_performance

In [ ]:
# Confusion table counts
confusion_matrix <- test_performance %>%
    table() 
confusion_matrix